In [125]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore", 'This pattern has match groups')
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.max_columns = 100000

## Viable Champions based on Past Champion Stats

In [76]:
tourney_teams_2023 = pd.read_csv('data/2023_team_results.csv')
tourney_teams_2023['Adj_OtoD_Margin'] = tourney_teams_2023['adjoe']-tourney_teams_2023['adjde']
teams_2023_final = pd.read_csv('data/2023_fffinal.csv')
teams_2023_final = teams_2023_final.rename(columns={"TeamName": "team"})
tourney_teams_2023_join = tourney_teams_2023.merge(teams_2023_final, on="team")
championship_candidates = tourney_teams_2023_join[(tourney_teams_2023_join["Adj_OtoD_Margin"] > 25) & (tourney_teams_2023_join["TO%"] < 18)]
championship_candidates.sort_values("Adj_OtoD_Margin", ascending=False)

,rank,team,conf,record,adjoe,oe Rank,adjde,de Rank,barthag,rank.1,proj. W,Proj. L,Pro Con W,Pro Con L,Con Rec.,sos,ncsos,consos,Proj. SOS,Proj. Noncon SOS,Proj. Con SOS,elite SOS,elite noncon SOS,Opp OE,Opp DE,Opp Proj. OE,Opp Proj DE,Con Adj OE,Con Adj DE,Qual O,Qual D,Qual Barthag,Qual Games,FUN,ConPF,ConPA,ConPoss,ConOE,ConDE,ConSOSRemain,Conf Win%,WAB,WAB Rk,"Fun Rk, adjt",Adj_OtoD_Margin,eFG%,eFG% Def,FTR,FTR Def,OR%,DR%,TO%,TO% Def.,3P%,3pD%,2p%,2p%D,ft%,ft%D,3P rate,3P rate D,arate,arateD
0,1,Houston,Amer,31-3,118.260139,12,89.970231,4,0.958679,1,31.955017,3.044983,17,1,17-1,0.616414,0.537336,0.654527,0.613721,0.537336,0.654527,0.768128,0.784425,107.504127,101.943296,107.378687,101.954088,122.010074,94.505982,123.465231,92.824828,0.963748,10,0.035108,1392,1104,1164.8625,1.194991,0.947751,0,0.944444,8.395558,5,106,28.289908,52.7,42.5,28.5,35.8,37.1,27.4,15.3,22.0,34.5,27.8,53.2,43.1,71.9,69.0,37.8,43.7,53.8,53.6
6,7,Gonzaga,WCC,28-5,125.199540,1,99.674033,80,0.932267,7,28.889635,5.110365,14,2,14-2,0.643860,0.612710,0.652126,0.643472,0.612710,0.652126,0.737805,0.718299,110.102984,101.838434,110.159501,101.991155,125.777731,101.370918,137.113806,105.506057,0.953180,10,0.056001,1431,1216,1153.7125,1.240344,1.053989,0,0.875000,7.074019,9,62,25.525507,58.5,51.7,34.2,26.7,31.1,26.0,14.6,18.8,38.4,35.0,59.0,51.2,70.2,73.5,32.8,39.4,50.7,45.3
5,6,Purdue,B10,29-5,119.494854,7,94.247513,21,0.938746,6,29.000000,5.000000,15,5,15-5,0.705650,0.544207,0.775853,0.705650,0.544207,0.775853,0.704254,0.780309,109.834133,100.430317,109.834133,100.430317,118.184136,93.928855,130.685688,102.677415,0.941245,15,0.062391,1418,1282,1274.2500,1.112811,1.006082,0,0.750000,9.178229,3,51,25.247341,52.2,47.2,37.2,19.3,38.3,23.3,17.0,15.4,32.6,31.4,54.1,47.4,74.3,73.6,38.1,35.4,60.9,51.5
2,3,UCLA,P12,29-5,113.602138,33,88.530852,2,0.946218,3,29.948633,5.051367,18,2,18-2,0.687179,0.531127,0.748903,0.681485,0.531127,0.748903,0.724026,0.805413,109.129526,100.286733,108.943759,100.366009,112.249097,87.574005,121.887739,96.794755,0.934063,14,0.034992,1400,1186,1311.5500,1.067439,0.904274,0,0.900000,8.439189,4,107,25.071286,50.9,46.8,27.1,27.6,33.7,27.6,15.3,23.6,34.7,31.1,50.4,46.8,72.2,69.7,29.1,39.3,50.9,51.3


## Old 2022 Simulator

In [77]:
#################################
#### MARCH MADNESS SIMULATOR ####
#################################

# Functions for running tournament simulator\n",
# Needs: High ADJOE and Low ADJDE (take care of this with margin), Low TOR\n",
# Wants: Low FTRD, High EFG, 3P_D, 2P_O\n",
# Other Notes...\n",
# Needs: ADJOE-ADJDE > 30, TOR < 16.5\n",
# Nice to have: FTRD < 31\n",
# Marg(0.61)up, TOR(0.14)*1.2down, Low FTRD(0.11)*.85down, Low 3P_D(0.08)down, High 2P_O(0.06)*0.5up\n",

def game_predictor(df, team1, team2):
    # Team 1 stats
    team1_df = df[df["team"].str.contains(team1)]
    team1_o2d_marg = team1_df["Adj_OtoD_Margin"]
    team1_TOR = team1_df["TO%"]
    team1_FTRD = team1_df["FTR Def"]
    team1_3PD = team1_df["3pD%"]
    team1_2PO = team1_df["2p%"]

    # Team 2 stats\n",
    team2_df = df[df["team"].str.contains(team2)]
    team2_o2d_marg = team2_df["Adj_OtoD_Margin"]
    team2_TOR = team2_df["TO%"]
    team2_FTRD = team2_df["FTR Def"]
    team2_3PD = team2_df["3pD%"]
    team2_2PO = team2_df["2p%"]

    #team1_total = team1_o2d_marg - (team1_TOR * 0.69)  #Initial equation
    team1_total = (team1_o2d_marg*0.61) - (team1_TOR*0.14*1.2) - (team1_FTRD*0.11*0.85) - (team1_3PD*0.08) + (team1_2PO*0.06*0.5)
    team1_total = team1_total.to_numpy()[0]
    #print(team1_total)
    #print(str(team1)+\" Total: \" + str(round(team1_total, 2)))
    #team2_total = team2_o2d_marg - (team2_TOR * 0.69)  #Initial equation
    team2_total = (team2_o2d_marg*0.61) - (team2_TOR*0.14*1.2) - (team2_FTRD*0.11*0.85) - (team2_3PD*0.08) + (team2_2PO*0.06*0.5)
    team2_total = team2_total.to_numpy()[0]
    #print(str(team2)+\" Total: \" + str(round(team2_total, 2)))

    result = team1_total > team2_total
    margin_of_pred = abs(team1_total - team2_total)
    if result == True:
        print("Winner: " + str(team1))
        return team1, margin_of_pred
    else:
        print("Winner: " + str(team2))
        return team2, margin_of_pred

def play_round(teams_remaining, stats_df):
    num_games = len(teams_remaining)//2
    print(num_games)
    winners_list = []
    margin_of_pred_list = []
    for game in range(0,num_games):
        team1_index = game*2
        team2_index = game*2+1
        team1_name = teams_remaining[team1_index]
        print("")
        print(team1_name)
        team2_name = teams_remaining[team2_index]
        print(team2_name)
        winner, margin_of_pred = game_predictor(stats_df,team1_name,team2_name)
        winners_list.append(winner)
        margin_of_pred_list.append(margin_of_pred)
    return winners_list, margin_of_pred_list

In [10]:
#Initialize Year
year = 2023

# Initialize Teams
df_round68 = ["Texas A&M Corpus Chris","Southeastern Louisiana","Texas Southern","Fairleigh Dickinson","Mississippi St.","Pittsburgh","Arizona St.","Nevada"]
df_round64 = ['Alabama','Texas A&M Corpus Chris','Maryland','West Virginia','San Diego St.','College of Charleston','Virginia','Furman',
              'Creighton','North Carolina St.','Baylor','UC Santa Barbara','Missouri','Utah St.','Arizona','Princeton',
              'Purdue','Texas Southern','Memphis','Florida Atlantic',"Duke",'Oral Roberts','Tennessee','Louisiana Lafayette','Kentucky','Providence',
              'Kansas St.','Montana St.','Michigan St.','USC','Marquette','Vermont',
              'Houston','Northern Kentucky','Iowa','Auburn','Miami FL','Drake','Indiana','Kent St.','Iowa St.','Mississippi St.',
              'Xavier','Kennesaw St.','Texas A&M','Penn St.','Texas','Colgate',
              'Kansas','Howard','Arkansas','Illinois',"Saint Mary's",'VCU','Connecticut','Iona','TCU','Nevada',
              'Gonzaga','Grand Canyon','Northwestern','Boise St.','UCLA','UNC Asheville']
df_round32 = []
df_round16 = []
df_round8 = []
df_round4 = []
df_round2 = []
df_round1 = []
margins64 = ["None here b/c no games played yet, go to next round"]
margins32 = []
margins16 = []
margins8 = []
margins4 = []
margins2 = []
margins1 = []

# Combine list of rounds
list_of_rounds = [df_round64,df_round32,df_round16,df_round8,df_round4,df_round2,df_round1]
pred_win_margins = [margins64,margins32,margins16,margins8,margins4,margins2,margins1]

# Play the tournament
print("This is March! Let the tournament begin...")
inv_round_counter = 5
for round in range(0,6):
    remaining_teams = 2**inv_round_counter
    winners, margins = play_round(list_of_rounds[round],tourney_teams_2023_join)
    list_of_rounds[round+1] = winners
    winners_df = pd.DataFrame(winners)
    #winners_df = winners_df.rename(index={0: "Remaining Teams"})
    win_csv_name = "sim_output/"+str(year)+"/roundof"+str(remaining_teams)+"_winners.csv"
    winners_df.to_csv(win_csv_name)
    pred_win_margins[round] = margins
    margins_df = pd.DataFrame(margins)
    #margins_df = margins_df.rename(index={0: "Predictor Score Margin of Victory"})
    margin_csv_name = "sim_output/"+str(year)+"/roundof"+str(remaining_teams)+"_margins.csv"
    margins_df.to_csv(margin_csv_name)
    inv_round_counter -= 1
champion = list_of_rounds[6][0]
print("")
print("")
print("***Congrats, your predicted champion is "+str(champion)+"!***")
print("For the results of all the rounds, check your file directory for each game's winner, or use the two cells below for quick access.")

This is March! Let the tournament begin...
32

Alabama
Texas A&M Corpus Chris
Winner: Alabama

Maryland
West Virginia
Winner: West Virginia

San Diego St.
College of Charleston
Winner: San Diego St.

Virginia
Furman
Winner: Virginia

Creighton
North Carolina St.
Winner: Creighton

Baylor
UC Santa Barbara
Winner: Baylor

Missouri
Utah St.
Winner: Utah St.

Arizona
Princeton
Winner: Arizona

Purdue
Texas Southern
Winner: Purdue

Memphis
Florida Atlantic
Winner: Memphis

Duke
Oral Roberts
Winner: Duke

Tennessee
Louisiana Lafayette
Winner: Tennessee

Kentucky
Providence
Winner: Kentucky

Kansas St.
Montana St.
Winner: Kansas St.

Michigan St.
USC
Winner: Michigan St.

Marquette
Vermont
Winner: Marquette

Houston
Northern Kentucky
Winner: Houston

Iowa
Auburn
Winner: Auburn

Miami FL
Drake
Winner: Miami FL

Indiana
Kent St.
Winner: Indiana

Iowa St.
Mississippi St.
Winner: Iowa St.

Xavier
Kennesaw St.
Winner: Xavier

Texas A&M
Penn St.
Winner: Texas A&M

Texas
Colgate
Winner: Texas

Kansa

# 2023 March Madness Simulator

In [266]:
# Functions for running tournament simulator\n",
# Needs: High ADJOE and Low ADJDE (take care of this with margin), Low TOR\n",
# Wants: Low FTRD, High EFG, 3P_D, 2P_O\n",
# Other Notes...\n",
# Needs: ADJOE-ADJDE > 30, TOR < 16.5\n",
# Nice to have: FTRD < 31\n",
# Marg(0.61)up, TOR(0.14)*1.2down, Low FTRD(0.11)*.85down, Low 3P_D(0.08)down, High 2P_O(0.06)*0.5up\n",

def stat_scaler(df, stat, inverse_stat=False):
    stat_min = df[stat].min()
    stat_max = df[stat].max()
    if inverse_stat == False:
        scaled_data = (df[stat]-stat_min)/(stat_max-stat_min) # Normalized stat (0-1 Min-Max Scaler)
        return scaled_data
    else: 
        scaled_data = 1-((df[stat]-stat_min)/(stat_max-stat_min))  # Inversed Normalized stat (0-1 Max Min Scaler)
        return scaled_data


def multi_stat_scaler(df, stat_weights, inverse_stat_flags):
    for stat in stat_weights.columns:
        inverse_flag = inverse_stat_flags[stat][0]
        if inverse_flag == False:
            df[stat] = stat_scaler(df, stat)
        else:
            df[stat] = stat_scaler(df, stat, inverse_stat=True)
    return df
    

def team_power_rating_generator(scaled_df, team, stat_weights):
    team_df = scaled_df[scaled_df["team"].str.contains(team)]
    team_pr = 0
    for stat in stat_weights:
        team_pr += team_df[stat].iloc[0] * stat_weights[stat].iloc[0]
    return team_pr


def game_predictor(scaled_df, stat_weights, team1, team2):
    # Team 1 Power Rating
    team1_pr = team_power_rating_generator(scaled_df, team1, stat_weights)

    # Team 2 Power Rating
    team2_pr = team_power_rating_generator(scaled_df, team2, stat_weights)

    result = team1_pr > team2_pr
    margin_of_pred = abs(team1_pr - team2_pr)
    if result == True:
        print("Winner: " + str(team1))
        return team1, margin_of_pred
    else:
        print("Winner: " + str(team2))
        return team2, margin_of_pred

def play_round(teams_remaining, stats_df, stat_weights):
    num_games = len(teams_remaining)//2
    print(num_games)
    winners_list = []
    margin_of_pred_list = []
    for game in range(0,num_games):
        team1_index = game*2
        team2_index = game*2+1
        team1_name = teams_remaining[team1_index]
        print("")
        print(team1_name)
        team2_name = teams_remaining[team2_index]
        print(team2_name)
        winner, margin_of_pred = game_predictor(stats_df,stat_weights,team1_name,team2_name)
        winners_list.append(winner)
        margin_of_pred_list.append(margin_of_pred)
    return winners_list, margin_of_pred_list

In [268]:
# Initialize Year
year = 2023

# User
username = "best_of_2500"

# Load Dataset
tourney_teams_2023 = pd.read_csv('data/2023_team_results.csv')
tourney_teams_2023['Adj_OtoD_Margin'] = tourney_teams_2023['adjoe']-tourney_teams_2023['adjde']
teams_2023_final = pd.read_csv('data/2023_fffinal.csv')
teams_2023_final = teams_2023_final.rename(columns={"TeamName": "team"})
tourney_teams_2023_join = tourney_teams_2023.merge(teams_2023_final, on="team")
championship_candidates = tourney_teams_2023_join[(tourney_teams_2023_join["Adj_OtoD_Margin"] > 25) & (tourney_teams_2023_join["TO%"] < 18)]
championship_candidates.sort_values("Adj_OtoD_Margin", ascending=False)

# Scale Tournament Data
stat_weights = pd.read_csv('model_weights/'+username+'_model_weights.csv')
inverse_stat_flags = pd.read_csv('model_weights/inverse_stat_flags.csv')
scaled_stat_df = multi_stat_scaler(tourney_teams_2023_join, stat_weights, inverse_stat_flags)

# Initialize Teams
df_round68 = ["Texas A&M Corpus Chris","Southeastern Louisiana","Texas Southern","Fairleigh Dickinson","Mississippi St.","Pittsburgh","Arizona St.","Nevada"]
df_round64 = ['Alabama','Texas A&M Corpus Chris','Maryland','West Virginia','San Diego St.','College of Charleston','Virginia','Furman',
              'Creighton','North Carolina St.','Baylor','UC Santa Barbara','Missouri','Utah St.','Arizona','Princeton',
              'Purdue','Texas Southern','Memphis','Florida Atlantic',"Duke",'Oral Roberts','Tennessee','Louisiana Lafayette','Kentucky','Providence',
              'Kansas St.','Montana St.','Michigan St.','USC','Marquette','Vermont',
              'Houston','Northern Kentucky','Iowa','Auburn','Miami FL','Drake','Indiana','Kent St.','Iowa St.','Pittsburgh',
              'Xavier','Kennesaw St.','Texas A&M','Penn St.','Texas','Colgate',
              'Kansas','Howard','Arkansas','Illinois',"Saint Mary's",'VCU','Connecticut','Iona','TCU','Nevada',
              'Gonzaga','Grand Canyon','Northwestern','Boise St.','UCLA','UNC Asheville']
df_round32 = []
df_round16 = []
df_round8 = []
df_round4 = []
df_round2 = []
df_round1 = []
margins64 = ["None here b/c no games played yet, go to next round"]
margins32 = []
margins16 = []
margins8 = []
margins4 = []
margins2 = []
margins1 = []
"""
# 2022 Teams
df_round64 = ['Gonzaga','Georgia St.','Boise St.','Memphis','Connecticut','New Mexico St.','Arkansas','Vermont','Alabama','Notre Dame','Texas Tech','Montana St.','Michigan St.','Davidson','Duke','Cal St. Fullerton','Baylor','Norfolk St.','North Carolina','Marquette',"Saint Mary's",'Indiana','UCLA','Akron','Texas','Virginia Tech','Purdue','Yale','Murray St.','San Francisco','Kentucky',"Saint Peter's",'Arizona','Wright St.','Seton Hall','TCU','Houston','UAB','Illinois','Chattanooga','Colorado St.','Michigan','Tennessee','Longwood','Ohio St.','Loyola Chicago','Villanova','Delaware','Kansas','Texas Southern','San Diego St.','Creighton','Iowa','Richmond','Providence','South Dakota St.','LSU','Iowa St.','Wisconsin','Colgate','USC','Miami (FL)','Auburn','Jacksonville St.']
df_round32 = []
df_round16 = []
df_round8 = []
df_round4 = []
df_round2 = []
df_round1 = []
margins64 = ["None here b/c no games played yet, go to next round"]
margins32 = []
margins16 = []
margins8 = []
margins4 = []
margins2 = []
margins1 = []
"""
# Combine list of rounds
list_of_rounds = [df_round64,df_round32,df_round16,df_round8,df_round4,df_round2,df_round1]
pred_win_margins = [margins64,margins32,margins16,margins8,margins4,margins2,margins1]

# Play the tournament
print("This is March! Let the tournament begin...")
inv_round_counter = 5
for round in range(0,6):
    remaining_teams = 2**inv_round_counter
    winners, margins = play_round(list_of_rounds[round],scaled_stat_df, stat_weights)
    list_of_rounds[round+1] = winners
    winners_df = pd.DataFrame(winners)
    #winners_df = winners_df.rename(index={0: "Remaining Teams"})
    win_csv_name = "sim_output/"+str(year)+"/"+str(username)+"/roundof"+str(remaining_teams)+"_winners.csv"
    winners_df.to_csv(win_csv_name)
    pred_win_margins[round] = margins
    margins_df = pd.DataFrame(margins)
    #margins_df = margins_df.rename(index={0: "Predictor Score Margin of Victory"})
    margin_csv_name = "sim_output/"+str(year)+"/"+str(username)+"/roundof"+str(remaining_teams)+"_margins.csv"
    margins_df.to_csv(margin_csv_name)
    inv_round_counter -= 1
champion = list_of_rounds[6][0]
print("")
print("")
print("***Congrats, your predicted champion is "+str(champion)+"!***")
print("For the results of all the rounds, check your file directory for each game's winner, or use the two cells below for quick access.")

#second_round_actuals = ['Gonzaga','Arkansas','Texas Tech','Duke','North Carolina','UCLA','Purdue', "Saint Peter's",'Arizona','Houston','Michigan','Villanova','Kansas', 'Providence', 'Iowa St.', 'Miami (FL)']
#y_preds = list_of_rounds[2]
#accuracy = accuracy_score(y_pred=y_preds, y_true=second_round_actuals)
#print(accuracy)

This is March! Let the tournament begin...
32

Alabama
Texas A&M Corpus Chris
Winner: Alabama

Maryland
West Virginia
Winner: Maryland

San Diego St.
College of Charleston
Winner: San Diego St.

Virginia
Furman
Winner: Furman

Creighton
North Carolina St.
Winner: North Carolina St.

Baylor
UC Santa Barbara
Winner: Baylor

Missouri
Utah St.
Winner: Utah St.

Arizona
Princeton
Winner: Arizona

Purdue
Texas Southern
Winner: Purdue

Memphis
Florida Atlantic
Winner: Memphis

Duke
Oral Roberts
Winner: Oral Roberts

Tennessee
Louisiana Lafayette
Winner: Tennessee

Kentucky
Providence
Winner: Kentucky

Kansas St.
Montana St.
Winner: Kansas St.

Michigan St.
USC
Winner: Michigan St.

Marquette
Vermont
Winner: Marquette

Houston
Northern Kentucky
Winner: Houston

Iowa
Auburn
Winner: Auburn

Miami FL
Drake
Winner: Drake

Indiana
Kent St.
Winner: Indiana

Iowa St.
Pittsburgh
Winner: Pittsburgh

Xavier
Kennesaw St.
Winner: Xavier

Texas A&M
Penn St.
Winner: Penn St.

Texas
Colgate
Winner: Texas

Ka

### Testing 1 on 1 Game Predictions

In [263]:
game_predictor(tourney_teams_2023_join,stat_weights,"Miami (FL)","Auburn")

Winner: Miami (FL)


C:\Users\mattg\AppData\Local\Temp\ipykernel_19256\1346493907.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  team_df = scaled_df[scaled_df["team"].str.contains(team)]


('Miami (FL)', 0.7037684670810336)

### View winners for a chosen round (Index 1-6 available)

In [22]:
list_of_rounds[1]

['Alabama',
 'West Virginia',
 'San Diego St.',
 'Virginia',
 'Creighton',
 'Baylor',
 'Utah St.',
 'Arizona',
 'Purdue',
 'Memphis',
 'Duke',
 'Tennessee',
 'Kentucky',
 'Kansas St.',
 'Michigan St.',
 'Marquette',
 'Houston',
 'Auburn',
 'Miami FL',
 'Indiana',
 'Iowa St.',
 'Xavier',
 'Texas A&M',
 'Texas',
 'Kansas',
 'Arkansas',
 "Saint Mary's",
 'Connecticut',
 'TCU',
 'Gonzaga',
 'Boise St.',
 'UCLA']

### View Model's Power Rating Margin between Winning and Losing Teams of Each Game

In [19]:
pred_win_margins[3]

[1.8209546108999994,
 2.8132028242000064,
 3.3861182791000006,
 1.5152153489999964]

### View Power Rating of a Chosen Team based on Personal Model

In [67]:
team_power_rating_generator(scaled_stat_df, 'North Carolina St.', stat_weights)

52.83041653681788

### Generate full D1 Power Rating Rankings based on Personal Model

In [72]:
pwr_rats = []
for team in scaled_stat_df['team']:
    pwr_rat = team_power_rating_generator(scaled_stat_df, team, stat_weights)
    pwr_rats.append(pwr_rat)

scaled_stat_df['pwr_rat'] = pwr_rats
scaled_stat_pwrrat_df = scaled_stat_df.sort_values('pwr_rat',ascending=False)
scaled_stat_pwrrat_df[['team','pwr_rat']].to_csv('sim_output/2023/original/team_power_ratings.csv')